In [1]:
%load_ext autoreload
%autoreload 2

import re
import glob
import lzma
import pickle
import pandas as pd
import numpy as np
import requests as r
import seaborn as sns
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from json import load, dump, loads, dumps
from joblib import hash
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestCentroid, RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import RidgeClassifier, PassiveAggressiveClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier

warnings.simplefilter('ignore')
mpl.style.use('ggplot')

## Source Data

IF source data is missing run Elasticsearch query to extract data and then save it in JSON format to `data` directory

In [9]:
# news_json = r.get('http://elasticsearch.storage:9200/indice/docs/_search?sort=date:desc&size=6000').json()
# with lzma.open('./data/news.json.gz', 'wt') as fh:
#   fh.write(dumps(news_json['hits']['hits']))

In [10]:
b = loads(lzma.open('./data/news.json.gz', 'rt').read())
df = pd.json_normalize(b)

## Common issues that we generally face during the data preparation phase:
 - Format and structure normalization
 - Detect and fix missing values
 - Duplicates removal
 - Units normalization
 - Constraints validations
 - Anomaly detection and removal
 - Study of features importance/relevance
 - Dimentional reduction, feature selection & extraction

In [11]:
df = df[['_source.body', '_source.date', '_source.subject', '_source.language', '_source.categories']]
df.columns = ['body', 'pubdate', 'subject', 'language', 'categories']

In [12]:
df.drop_duplicates(inplace=True)
df.head(1).T.style

,0
body,"Islam Times - The President described the development of relations between Tehran and Dushanbe in the last 2 years as favorable, stated that the development of parliamentary cooperation is important and the basis of political and economic cooperation, he also emphasized the strengthening and promotion of cultural relations between the two countries."
pubdate,2024-01-08T19:10:23.000Z
subject,"Raisi: ISIS, Like Zionists in Gaza, Seeks to Kill Innocent Women and Children"
language,English
categories,Iran


In [13]:
df = df.loc[(df['categories'] != 'News') & 
            (df['categories'] != 'articles 2015') & 
            (df['categories'] != 'frontpage') &
            (df['categories'] != 'English') &
            (df['categories'] != 'Comment') &
            (df['categories'] != 'Uncategorized') &
            (df['language'] == 'English')]

In [14]:
df['categories'] = df['categories'].str.replace(r'[^a-zA-Z_, ]+', '').replace(', ', '')
df['categories'] = df['categories'].str.replace(r'^, ', '')
df = df[~df.eq('').all(1)]

In [15]:
df.groupby(['categories']).agg({'count'}).drop_duplicates()

,body,pubdate,subject,language
,count,count,count,count
categories,,,,
"#ACFM, Audio, 90s",1,1,1,1
"#Culture, featured, Featured Culture",2,2,2,2
"2021, Honest Ad",6,6,6,6
"2021, Teacher",4,4,4,4
2022,3,3,3,3
"Afghanistan, Article, Asia & Americas",7,7,7,7
"Afghanistan, Asia & Americas, Europe & Russia",13,13,13,13
"Africa, Algeria, Asia & Americas",8,8,8,8


In [16]:
df['cat_id']       = df['categories'].factorize()[0]
df['lang_id']      = df['language'].factorize()[0]
df['char_count']   = df['body'].apply(len)
df['word_count']   = df['body'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)


In [ ]:
df.isnall().sum()

In [24]:
df = df.dropna()

In [25]:
df.shape

(5402, 10)

In [26]:
# sns.set()
# sns.pairplot(df, height=3.5, kind="reg", palette="husl", diag_kind="auto")

In [27]:
xtrain, xtest, ytrain, ytest = train_test_split(df['body'], df['categories'], test_size=0.3, random_state=20)

In [28]:

tfidf = TfidfVectorizer(use_idf=False, sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', analyzer='char', lowercase=False, ngram_range=(1, 2), stop_words='english')

In [29]:
features = tfidf.fit_transform(df.body).toarray()
labels = df.cat_id

In [30]:
engines = [('PassiveAggressiveClassifier', PassiveAggressiveClassifier(fit_intercept=True)),
           ('NearestCentroid', NearestCentroid()),
           ('RandomForestClassifier', RandomForestClassifier(min_samples_leaf=0.01)),
           ('RidgeClassifierCV', RidgeClassifier(fit_intercept=True)),
           ('RadiusNeighborsClassifier', RadiusNeighborsClassifier()),
           ('DecisionTreeClassifier', DecisionTreeClassifier(min_samples_leaf=0.01))]

In [31]:
for name, engine in engines:
  try:
    clf        = make_pipeline(tfidf, engine).fit(xtrain, ytrain)
    prediction = clf.predict(xtest)
    score      = clf.score(xtest, prediction)
    with lzma.open('./data/{}.pickle.xz'.format(name.lower()), 'wb') as f:
      pickle.dump(clf, f, protocol=5)
  except Exception as e:
    print("Epic fail on {}, error: {}".format(engine, e))

In [32]:
s = '''

‘Guys, you’ve got to hear this,” I said. I was sitting in front of my computer one day in July 2012, with one eye on a screen of share prices and the other on a live stream of the House of Commons Treasury select committee hearings. As the Barclays share price took a graceful swan dive, I pulled my headphones out of the socket and turned up the volume so everyone could hear. My colleagues left their terminals and came around to watch BBC Parliament with me.

It didn’t take long to realise what was happening. “Bob’s getting murdered,” someone said.

Bob Diamond, the swashbuckling chief executive of Barclays, had been called before the committee to explain exactly what his bank had been playing at in regards to the Libor rate-fixing scandal. The day before his appearance, he had made things very much worse by seeming to accuse the deputy governor of the Bank of England of ordering him to fiddle an important benchmark, then walking back the accusation as soon as it was challenged. He was trying to turn on his legendary charm in front of a committee of angry MPs, and it wasn’t working. On our trading floor, in Mayfair, calls were coming in from all over the City. Investors needed to know what was happening and whether the damage was reparable.

A couple of weeks later, the damage was done. The money was gone, Diamond was out of a job and the market, as it always does, had moved on. We were left asking ourselves: How did we get it so wrong?

'''

In [33]:
result = []
for file in glob.glob('./data/*.pickle.xz'):
  clf   = pickle.load(lzma.open('{}'.format(file), 'rb'))
  ypred = clf.predict([s])
  score = clf.score([s], ypred)
  print(file, ypred[0], score)
  result.append(ypred[0])

print(pd.io.json.dumps(Counter(result), indent=4))

./data/ridgeclassifiercv.pickle.xz Opinion 1.0
./data/nearestcentroid.pickle.xz Opinion 1.0
./data/decisiontreeclassifier.pickle.xz Tech 1.0
./data/radiusneighborsclassifier.pickle.xz Israel, Middle East, News 1.0
./data/passiveaggressiveclassifier.pickle.xz Opinion 1.0
./data/randomforestclassifier.pickle.xz Opinion 1.0
{
    "Opinion":4,
    "Tech":1,
    "Israel, Middle East, News":1
}
